**Neural Networks Hypothesis Testing before/after Feature Selection**

In this notebook, we're gonna do the hypothesis testing before and after feature selection, using neural networks.
We will present the training and everything about the NN models we used earlier (so first part is gonna be the same code to recall old results), and then do the hypothesis testing one before and after models, after we train them on the data before and after feature selection. 
Also recall that using neural networks with FS (or without) had good accuracy but very bda fscore, and we don't consider this as good result so it's not actually the best model.

In [1]:
import numpy as np
import pandas as pd
import random as rd
import pickle 
from sklearn.feature_selection import SelectKBest, chi2 ,f_classif
!pip install learn2learn
import learn2learn as l2l
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.model_selection import train_test_split, learning_curve, ShuffleSplit, GridSearchCV
from sklearn.metrics import accuracy_score 
from torch import nn, optim
from sklearn.metrics import f1_score, log_loss, roc_auc_score
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
import torch
!pip install skorch
from skorch.classifier import NeuralNetClassifier

     |████████████████████████████████| 1.4MB 5.6MB/s 
     |████████████████████████████████| 2.5MB 17.7MB/s 
     |████████████████████████████████| 174kB 44.5MB/s 
     |████████████████████████████████| 102kB 10.2MB/s 
     |████████████████████████████████| 61kB 7.9MB/s 
     |████████████████████████████████| 61kB 3.7MB/s 
     |████████████████████████████████| 1.4MB 40.7MB/s 
     |████████████████████████████████| 112kB 45.2MB/s 
     |████████████████████████████████| 2.6MB 46.6MB/s 
  Created wheel for learn2learn: filename=learn2learn-0.1.5-cp36-cp36m-linux_x86_64.whl size=915668 sha256=0589d62895687ca7a08e16867df6d555ef6c53d057b13e8360ca98671b01befc
  Stored in directory: /root/.cache/pip/wheels/5a/44/11/9d755af86d74f353dc7e56d5e75668e4ae7f5bf8c7d74f1f7e
  Created wheel for gsutil: filename=gsutil-4.57-cp36-none-any.whl size=3340433 sha256=538c3d3a2ffac54f617b3f663421dab7e51169d75713dfe51ff608eb86251160
  Stored in directory: /root/.cache/pip/wheels/71/18/1c/9f12d053973060

In [2]:
# given features matrix and array with important feature indexes, this function returns the selected features
def get_selected_features(X, f_i):
  X_selected = []
  for x in X:
    selected = []
    for i in f_i:
      selected.append(x[i])
    X_selected.append(selected)
  return np.array(X_selected)

# same function but this is woring with dataframes instead of numpy arrays, returns a dataframe with chosen features
def get_selected_features_df(X_df, f_i):
  X_selected = X_df.copy()
  i = 0
  for col in X_df.iloc[:, 4:]:
    if i not in f_i: # numerical starts with 4
      X_selected.pop(col) # drop feature column at index i 
    i += 1
  return X_selected


In [3]:
# load data - pre-processed before, we just need to load feature selection tools to use them (feature indexes, data...)
fi = []
train_data = []
with open('/content/drive/My Drive/Colab Notebooks/features_indexes.data', 'rb') as f:
  fi = pickle.load(f) 

# train_data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/X_afterSteps_12.csv') 
train_data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/train_features.csv') 
df_y = pd.read_csv("/content/drive/My Drive/Colab Notebooks/train_targets_scored.csv")
train_labels = np.array(df_y.iloc[:, 1:])

In [4]:
x = train_data.iloc[:, 2:]

def encode(df, col):
  new_df = df.copy()
  classes = list(pd.unique(new_df.iloc[:,col]))
  for i in range(len(df.iloc[:,col])):
    new_df.iloc[i, col] = classes.index(new_df.iloc[i, col])
  return new_df

x=encode(x,0)
x=encode(x,1) 
x=encode(x,2)
print(x)
y = df_y.iloc[:, 1:]
X = x

       cp_time cp_dose      g-0     g-1  ...    c-96    c-97    c-98    c-99
0            0       0      0.0  0.5577  ... -0.3981  0.2139  0.3801  0.4176
1            1       0      1.0  0.4087  ...  0.1522  0.1241  0.6077  0.7371
2            2       0      2.0  0.5817  ... -0.6417 -0.2187 -1.4080  0.6931
3            2       0      3.0 -0.2491  ... -1.6210 -0.8784 -0.3876 -0.8154
4            1       1      4.0 -0.4009  ...  0.1094  0.2885 -0.3786  0.7125
...        ...     ...      ...     ...  ...     ...     ...     ...     ...
23809        0       1    527.0 -0.0636  ...  0.0631  0.9171  0.5258  0.4680
23810        0       1   1138.0  0.3478  ... -0.2084 -0.1224 -0.2715  0.3689
23811        2       1  14364.0  0.3756  ...  0.2256  0.7592  0.6656  0.3808
23812        0       0  14365.0  0.2324  ...  0.1732  0.7015 -0.6290  0.0740
23813        1       0  14366.0  1.0240  ... -3.5770 -0.4775 -2.1500 -4.2520

[23814 rows x 874 columns]


**NN before Feature Selection Code**: old code just to show and recall results on NN

In [6]:
import torch.nn as nn
import torch.nn.functional as F

# split data 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

temp = []
for i in X_train:
    temp.append(i) 
X_train_tensor = torch.Tensor(temp)

temp = []
for i in X_test:
  temp.append(i)
X_test_tensor = torch.Tensor(temp)
y_train_tensor = torch.Tensor(y_train)
y_test_tensor = torch.Tensor(y_test)


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(X.shape[1], 256)  
        self.fc2 = nn.Linear(256, 1024) 
        self.fc3 = nn.Linear(1024, 512) 
        self.fc4 = nn.Linear(512, 206) 

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(nn.Dropout(p=0.2)(self.fc2(x)))
        x = F.relu(nn.Dropout(p=0.2)(self.fc3(x)))
        x = self.fc4(x)
        return torch.sigmoid(x)

nn_noFS= Net()
print(nn_noFS)

Net(
  (fc1): Linear(in_features=874, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=1024, bias=True)
  (fc3): Linear(in_features=1024, out_features=512, bias=True)
  (fc4): Linear(in_features=512, out_features=206, bias=True)
)


In [7]:
# Pure deep NN

X_train = X_train_tensor
y_train = y_train_tensor
X_test = X_test_tensor
y_test = y_test_tensor

criterion = nn.BCELoss()
optimizer = optim.SGD(nn_noFS.parameters(), lr=0.001)

def round_tensor(t, decimal_places=3):
  return round(t.item(), decimal_places)

def calculate_accuracy(y_true, y_pred):
  predicted = y_pred.ge(.5).view(-1) # convert <0.5 to 0 and >0.5 to 1 (this threshold we can change it later)
  y_true = y_true.reshape((y_true.shape[0]*y_true.shape[1],))
  return (y_true == predicted).sum().float() / len(y_true)

for epoch in range(300):
  y_pred = nn_noFS(X_train)
  y_pred = torch.squeeze(y_pred)
  train_loss = criterion(y_pred, y_train)
  if epoch % 100 == 0:
    train_acc = calculate_accuracy(y_train, y_pred)
    y_test_pred = nn_noFS(X_test)
    y_test_pred = torch.squeeze(y_test_pred)
    test_loss = criterion(y_test_pred, y_test)
    test_acc = calculate_accuracy(y_test, y_test_pred)
    print(
f'''epoch {epoch}
Train set - loss: {round_tensor(train_loss)}, accuracy: {round_tensor(train_acc)}
Test  set - loss: {round_tensor(test_loss)}, accuracy: {round_tensor(test_acc)}
''')
  optimizer.zero_grad()
  train_loss.backward()
  optimizer.step()

epoch 0
Train set - loss: 11.385, accuracy: 0.509
Test  set - loss: 11.456, accuracy: 0.509

epoch 100
Train set - loss: 0.237, accuracy: 0.937
Test  set - loss: 0.237, accuracy: 0.936

epoch 200
Train set - loss: 0.144, accuracy: 0.975
Test  set - loss: 0.144, accuracy: 0.975



In [8]:
outputs = nn_noFS(X_train_tensor)
print("Log loss: " + str(log_loss(y_train_tensor.detach().numpy(), outputs.detach().numpy())))

outputs[outputs >= 0.5] = 1 
outputs[outputs < 0.5] = 0 
accuracy = (outputs == y_train_tensor).sum() / (y_train_tensor.shape[0]*y_train_tensor.shape[1]) * 100
print("Accuracy: " + str(accuracy) + "%")  

f1 = f1_score(y_train_tensor.detach().numpy(), outputs.detach().numpy(), average='weighted')
print("f1-score: " + str(f1))

Log loss: 7.874820894489429
Accuracy: tensor(98.5397)%
f1-score: 0.01113079059574135


Again a small note, even though the accuracy is high, neural network had a very bad fscore in our project so it isn't really any better than other things we tried such as SVM

**NN after Feature Selection Code**: again old code, no testing yet

In [9]:
train_data = get_selected_features_df(train_data, fi)

In [10]:
x = train_data.iloc[:, 2:]

def encode(df, col):
  new_df = df.copy()
  classes = list(pd.unique(new_df.iloc[:,col]))
  for i in range(len(df.iloc[:,col])):
    new_df.iloc[i, col] = classes.index(new_df.iloc[i, col])
  return new_df

x=encode(x,0)
x=encode(x,1) 
x=encode(x,2)
print(x)
y = df_y.iloc[:, 1:]
X = x

       cp_time cp_dose      g-0     g-7  ...    c-96    c-97    c-98    c-99
0            0       0      0.0 -0.0326  ... -0.3981  0.2139  0.3801  0.4176
1            1       0      1.0  0.3372  ...  0.1522  0.1241  0.6077  0.7371
2            2       0      2.0  0.2155  ... -0.6417 -0.2187 -1.4080  0.6931
3            2       0      3.0  0.1792  ... -1.6210 -0.8784 -0.3876 -0.8154
4            1       1      4.0 -0.1498  ...  0.1094  0.2885 -0.3786  0.7125
...        ...     ...      ...     ...  ...     ...     ...     ...     ...
23809        0       1    527.0  0.3055  ...  0.0631  0.9171  0.5258  0.4680
23810        0       1   1138.0 -0.5565  ... -0.2084 -0.1224 -0.2715  0.3689
23811        2       1  14364.0  0.1745  ...  0.2256  0.7592  0.6656  0.3808
23812        0       0  14365.0  0.0463  ...  0.1732  0.7015 -0.6290  0.0740
23813        1       0  14366.0  0.9146  ... -3.5770 -0.4775 -2.1500 -4.2520

[23814 rows x 395 columns]


In [11]:
import torch.nn as nn
import torch.nn.functional as F

# split data 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

temp = []
for i in X_train:
    temp.append(i) 
X_train_tensor = torch.Tensor(temp)

temp = []
for i in X_test:
  temp.append(i)
X_test_tensor = torch.Tensor(temp)
y_train_tensor = torch.Tensor(y_train)
y_test_tensor = torch.Tensor(y_test)

nn_FS= Net()
print(nn_FS)

Net(
  (fc1): Linear(in_features=395, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=1024, bias=True)
  (fc3): Linear(in_features=1024, out_features=512, bias=True)
  (fc4): Linear(in_features=512, out_features=206, bias=True)
)


In [12]:
X_train = X_train_tensor
y_train = y_train_tensor
X_test = X_test_tensor
y_test = y_test_tensor

criterion = nn.BCELoss()
optimizer = optim.SGD(nn_FS.parameters(), lr=0.001)

def round_tensor(t, decimal_places=3):
  return round(t.item(), decimal_places)

def calculate_accuracy(y_true, y_pred):
  predicted = y_pred.ge(.5).view(-1) # convert <0.5 to 0 and >0.5 to 1 (this threshold we can change it later)
  y_true = y_true.reshape((y_true.shape[0]*y_true.shape[1],))
  return (y_true == predicted).sum().float() / len(y_true)

for epoch in range(300):
  y_pred = nn_FS(X_train)
  y_pred = torch.squeeze(y_pred)
  train_loss = criterion(y_pred, y_train)
  if epoch % 100 == 0:
    train_acc = calculate_accuracy(y_train, y_pred)
    y_test_pred = nn_FS(X_test)
    y_test_pred = torch.squeeze(y_test_pred)
    test_loss = criterion(y_test_pred, y_test)
    test_acc = calculate_accuracy(y_test, y_test_pred)
    print(
f'''epoch {epoch}
Train set - loss: {round_tensor(train_loss)}, accuracy: {round_tensor(train_acc)}
Test  set - loss: {round_tensor(test_loss)}, accuracy: {round_tensor(test_acc)}
''')
  optimizer.zero_grad()
  train_loss.backward()
  optimizer.step()

epoch 0
Train set - loss: 17.003, accuracy: 0.486
Test  set - loss: 17.011, accuracy: 0.486

epoch 100
Train set - loss: 0.173, accuracy: 0.97
Test  set - loss: 0.174, accuracy: 0.97

epoch 200
Train set - loss: 0.123, accuracy: 0.985
Test  set - loss: 0.124, accuracy: 0.985



In [13]:
outputs = nn_FS(X_train_tensor)
print("Log loss: " + str(log_loss(y_train_tensor.detach().numpy(), outputs.detach().numpy())))

outputs[outputs >= 0.5] = 1 
outputs[outputs < 0.5] = 0 
accuracy = (outputs == y_train_tensor).sum() / (y_train_tensor.shape[0]*y_train_tensor.shape[1]) * 100
print("Accuracy: " + str(accuracy) + "%")  

f1 = f1_score(y_train_tensor.detach().numpy(), outputs.detach().numpy(), average='weighted')
print("f1-score: " + str(f1))

Log loss: 10.185718847707866
Accuracy: tensor(98.9180)%
f1-score: 0.011261332635926246


In [ ]:
# save models
with open("/content/drive/My Drive/Colab Notebooks/nn_noFS.sav", 'wb') as f:
  pickle.dump(nn_noFS, f)
with open("/content/drive/My Drive/Colab Notebooks/nn_FS.sav", 'wb') as f:
  pickle.dump(nn_FS, f)

Slightly worse results than without FS (logloss), better accuracy though that does not mean a better model

**Compare: Hypothesis Testing**

CV Scores for NO_FS (no feature selection)

In [32]:
# CV for no_FS (no feature seelction)

# load data without FS
fi = []
train_data = []
train_data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/train_features.csv') 
df_y = pd.read_csv("/content/drive/My Drive/Colab Notebooks/train_targets_scored.csv")
train_labels = np.array(df_y.iloc[:, 1:])
x = train_data.iloc[:, 2:]

def encode(df, col):
  new_df = df.copy()
  classes = list(pd.unique(new_df.iloc[:,col]))
  for i in range(len(df.iloc[:,col])):
    new_df.iloc[i, col] = classes.index(new_df.iloc[i, col])
  return new_df

x=encode(x,0)
x=encode(x,1) 
x=encode(x,2)
print(x)
y = df_y.iloc[:, 1:]
X = x
X_train = np.array(X)
y_train = np.array(y)
temp = []
for i in X_train:
    temp.append(i) 
X_train_tensor = torch.Tensor(temp)
y_train_tensor = torch.Tensor(y_train)
X_train = X_train_tensor
y_train = y_train_tensor  

criterion = nn.BCELoss()
optimizer = optim.SGD(nn_FS.parameters(), lr=0.001)

# model
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(X.shape[1], 256)  
        self.fc2 = nn.Linear(256, 1024) 
        self.fc3 = nn.Linear(1024, 512) 
        self.fc4 = nn.Linear(512, 206) 

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(nn.Dropout(p=0.2)(self.fc2(x)))
        x = F.relu(nn.Dropout(p=0.2)(self.fc3(x)))
        x = self.fc4(x)
        return torch.sigmoid(x)

nn_noFS= Net()
print(nn_noFS)

       cp_time cp_dose      g-0     g-1  ...    c-96    c-97    c-98    c-99
0            0       0      0.0  0.5577  ... -0.3981  0.2139  0.3801  0.4176
1            1       0      1.0  0.4087  ...  0.1522  0.1241  0.6077  0.7371
2            2       0      2.0  0.5817  ... -0.6417 -0.2187 -1.4080  0.6931
3            2       0      3.0 -0.2491  ... -1.6210 -0.8784 -0.3876 -0.8154
4            1       1      4.0 -0.4009  ...  0.1094  0.2885 -0.3786  0.7125
...        ...     ...      ...     ...  ...     ...     ...     ...     ...
23809        0       1    527.0 -0.0636  ...  0.0631  0.9171  0.5258  0.4680
23810        0       1   1138.0  0.3478  ... -0.2084 -0.1224 -0.2715  0.3689
23811        2       1  14364.0  0.3756  ...  0.2256  0.7592  0.6656  0.3808
23812        0       0  14365.0  0.2324  ...  0.1732  0.7015 -0.6290  0.0740
23813        1       0  14366.0  1.0240  ... -3.5770 -0.4775 -2.1500 -4.2520

[23814 rows x 874 columns]
Net(
  (fc1): Linear(in_features=874, out_featur

In [33]:
# since i was using pytorch for this part, i implemented my own k-fold, i was having problems with pytorch interacting with sklearn library

# returns train indices
def StratifiedKFold(labels, n_splits):
  train = []
  n = len(labels)//n_splits
  for i in range(n_splits):
    train.append((n*i, n*(i+1)-1))
  return train

n_folds = 3
data, labels = X_train, y_train
skf = StratifiedKFold(labels, n_splits=n_folds) 

In [40]:
# get CV scores - 100 epochs, reduced just because it was taking much time, results should be fair though since after 100 epochs we see it converges (not necessarily better with more epochs)
scores_no_FS = []

k = 0
for (i, j) in skf:
  k += 1
  print("Running Fold", k, "/", n_folds)
  nn_noFS = None 
  nn_noFS = Net() 
  optimizer = optim.SGD(nn_noFS.parameters(), lr=0.001)

  X_train = data[i:j]
  y_train = labels[i:j]

  for epoch in range(100):
    y_pred = nn_noFS(X_train)
    y_pred = torch.squeeze(y_pred)
    train_loss = criterion(y_pred, y_train)
    # if epoch % 100 == 0:
      # y_test_pred = nn_noFS(X_test)
      # y_test_pred = torch.squeeze(y_test_pred)
      # test_loss = criterion(y_test_pred, y_test)
      # test_acc = calculate_accuracy(y_test, y_test_pred)
    optimizer.zero_grad()
    train_loss.backward()
    optimizer.step()

  train_acc = calculate_accuracy(y_train, y_pred)
  scores_no_FS.append(train_acc)
  print("Score for this fold: " + str(train_acc))

Running Fold 1 / 3
Score for this fold: tensor(0.8947)
Running Fold 2 / 3
Score for this fold: tensor(0.9251)
Running Fold 3 / 3
Score for this fold: tensor(0.9417)


CV Scores for FS (Feature Selection)

In [41]:
# load data with FS
fi = []
train_data = []
train_data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/train_features.csv') 
df_y = pd.read_csv("/content/drive/My Drive/Colab Notebooks/train_targets_scored.csv")
train_labels = np.array(df_y.iloc[:, 1:])
fi = []
with open('/content/drive/My Drive/Colab Notebooks/features_indexes.data', 'rb') as f:
  fi = pickle.load(f) 
train_data = get_selected_features_df(train_data, fi) # select features
x = train_data.iloc[:, 2:]

def encode(df, col):
  new_df = df.copy()
  classes = list(pd.unique(new_df.iloc[:,col]))
  for i in range(len(df.iloc[:,col])):
    new_df.iloc[i, col] = classes.index(new_df.iloc[i, col])
  return new_df

x=encode(x,0)
x=encode(x,1) 
x=encode(x,2)
print(x)
y = df_y.iloc[:, 1:]
X = x
X_train = np.array(X)
y_train = np.array(y)
temp = []
for i in X_train:
    temp.append(i) 
X_train_tensor = torch.Tensor(temp)
y_train_tensor = torch.Tensor(y_train)
X_train = X_train_tensor
y_train = y_train_tensor  

criterion = nn.BCELoss()
optimizer = optim.SGD(nn_FS.parameters(), lr=0.001)

       cp_time cp_dose      g-0     g-7  ...    c-96    c-97    c-98    c-99
0            0       0      0.0 -0.0326  ... -0.3981  0.2139  0.3801  0.4176
1            1       0      1.0  0.3372  ...  0.1522  0.1241  0.6077  0.7371
2            2       0      2.0  0.2155  ... -0.6417 -0.2187 -1.4080  0.6931
3            2       0      3.0  0.1792  ... -1.6210 -0.8784 -0.3876 -0.8154
4            1       1      4.0 -0.1498  ...  0.1094  0.2885 -0.3786  0.7125
...        ...     ...      ...     ...  ...     ...     ...     ...     ...
23809        0       1    527.0  0.3055  ...  0.0631  0.9171  0.5258  0.4680
23810        0       1   1138.0 -0.5565  ... -0.2084 -0.1224 -0.2715  0.3689
23811        2       1  14364.0  0.1745  ...  0.2256  0.7592  0.6656  0.3808
23812        0       0  14365.0  0.0463  ...  0.1732  0.7015 -0.6290  0.0740
23813        1       0  14366.0  0.9146  ... -3.5770 -0.4775 -2.1500 -4.2520

[23814 rows x 395 columns]


In [44]:
n_folds = 3
data, labels = X_train_tensor, y_train_tensor
skf = StratifiedKFold(labels, n_splits=n_folds) 
scores_FS = []

k = 0
for (i, j) in skf:
  k += 1
  print("Running Fold", k, "/", n_folds)
  nn_noFS = None 
  nn_noFS = Net() 
  optimizer = optim.SGD(nn_noFS.parameters(), lr=0.001)

  X_train = data[i:j]
  y_train = labels[i:j]

  for epoch in range(100):
    y_pred = nn_noFS(X_train)
    y_pred = torch.squeeze(y_pred)
    train_loss = criterion(y_pred, y_train)
    # if epoch % 100 == 0:
      # y_test_pred = nn_noFS(X_test)
      # y_test_pred = torch.squeeze(y_test_pred)
      # test_loss = criterion(y_test_pred, y_test)
      # test_acc = calculate_accuracy(y_test, y_test_pred)
    optimizer.zero_grad()
    train_loss.backward()
    optimizer.step()

  train_acc = calculate_accuracy(y_train, y_pred)
  scores_FS.append(train_acc)
  print("Score for this fold: " + str(train_acc))

Running Fold 1 / 3
Score for this fold: tensor(0.9380)
Running Fold 2 / 3
Score for this fold: tensor(0.9689)
Running Fold 3 / 3
Score for this fold: tensor(0.9763)


Compare

In [45]:
from scipy.stats import ttest_ind

t_stat, p = ttest_ind(scores_no_FS, scores_FS)

print(f'The P-value is = {p:.3f}')

if p <= 0.05:
    print('Since p < 0.05, rejecting the null-hypothesis that both models perform the same. The two models are significantly different.')
else:
    print('Since p > 0.05, we cannot reject the null hypothesis, we conclude that the performance of the two models is not significantly different.')

The P-value is = 0.088
Since p > 0.05, we cannot reject the null hypothesis, we conclude that the performance of the two models is not significantly different.
